In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import torch
import sys
from helpers_task_detector import *
sys.path.append('../')
from src.helpers import *
from src.visualize import *
from src.trainer import *
from src.trainer_hnet import * 
from Models.models import *
import json

import seaborn as sns
import matplotlib.pyplot as plt

/home/nerea/anaconda3/envs/sinthlab/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the list of experiments from JSON
with open(os.path.join('config.json'), 'r') as f:
    experiments = json.load(f)
    
path_to_hnets = './Models/Models_HNET_Block3'

In [3]:
experiment0_hnet_v0 = 'Experiment1_Task3'
models_exp0_hnet_v0 = np.sort(os.listdir(os.path.join(path_to_hnets, experiment0_hnet_v0)))


path_results_exp0_v0 = './Results/' + experiment0_hnet_v0
with open(os.path.join(path_results_exp0_v0 +'.pkl'), 'rb') as fp:
    results_hnet_exp0_v0 = pickle.load(fp)

data_path_exp0_v0 = './Data/Sim_Data_Experiment60'+'.pkl'
with open(os.path.join(data_path_exp0_v0), 'rb') as fp:
    data_exp0_v0 = pickle.load(fp)

In [4]:
model1 = torch.load(os.path.join(path_to_hnets,experiment0_hnet_v0, models_exp0_hnet_v0[1] ))
model2 = torch.load(os.path.join(path_to_hnets,experiment0_hnet_v0, models_exp0_hnet_v0[2] ))
model9 = torch.load(os.path.join(path_to_hnets,experiment0_hnet_v0, models_exp0_hnet_v0[7] ))

In [5]:
first_task = list(data_exp0_v0.keys())[1]

In [6]:
x_train, y_train, x_val, y_val, x_test, y_test = data_exp0_v0[first_task]
W_1 = model1(cond_id = 1)
W_2 = model2(cond_id = 1)
W_9 = model9(cond_id = 1)

In [7]:
main_net = RNN_Main_Model(num_features= 130, hnet_output = W_1,  hidden_size = 300,
                                        num_layers= 1,out_dims=2,  
                                        dropout= 0.2,  LSTM_ = False)

In [8]:
r2_1, _ = calc_explained_variance_mnet(x_test, y_test, W_1, main_net)

In [9]:
r2_1

0.8618788421154022

In [10]:
r2_2, _ = calc_explained_variance_mnet(x_test, y_test, W_2, main_net)


In [11]:
r2_2

0.7896788120269775

In [12]:
r2_9, _ = calc_explained_variance_mnet(x_test, y_test, W_9, main_net)

In [13]:
r2_9

0.8244554698467255

In [19]:
task_performances = {}
best_perf = {}
for d in data_exp0_v0.keys():
    x_train, y_train, x_val, y_val, x_test, y_test = data_exp0_v0[d]
    data_perf = []
    model = torch.load(os.path.join(path_to_hnets,experiment0_hnet_v0, models_exp0_hnet_v0[-1] ))
    for cond in range(len(models_exp0_hnet_v0)):
        W = model(cond_id = cond)
        r2_, _ = calc_explained_variance_mnet(x_test, y_test, W, main_net)
        data_perf.append(r2_)
    task_performances[d] = data_perf
    best_perf[d] = max(data_perf)


In [21]:
best_perf

{'Data_1_2': 0.7641560733318329,
 'Data_2_2': 0.8220682442188263,
 'Data_4_1': 0.6173930764198303,
 'Data_4_2': 0.6004714369773865,
 'Data_1_1': 0.7592190504074097,
 'Data_2_1': 0.8050783276557922,
 'Data_0_1': 0.7833098769187927,
 'Data_3_1': 0.7298378050327301,
 'Data_3_2': 0.7538323700428009,
 'Data_0_2': 0.796068549156189}